# CEE 609 Term Project - Data Download and Pre-processing Code
## Adarsh Raghuram
This study examines changes in sensitivity of corn yields to heat stress in the rainfed regions of the United States. Datasets used include climate renanlysis data from ERA5, annual county-level yield data from USDA and crop planting and harvesting dates from Sachs et al. (2010). Methods for obtaining and processing data in explained in the following sections. Unless indicated otherwise, the codes are written in R Programming Language Version 4.1.1 

## 1. Downloading Shapefile from US Census Database
I have used an older (2010) shapefile for mapping yields with counties, however, newer ones are availabe. The following link contains the archival and latest shapefiles - https://www2.census.gov/geo/tiger/GENZ2010/ 

In [4]:
shp <- download.file('https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_us_050_00_20m.zip', 'us_shp.zip')
zipPath <- "/downloads/us_shp.zip" # Path for the zip file
dir.create('/downloads/us_shp')
out <- "/downloads/us_shp"
unzip(zipPath,out) # Unzip the zipped files

In [ ]:
# Download Crop Planting and Harvesting Dates
dates <- download.file('https://drive.google.com/file/d/1jVbKPigKbz9i6mwonza3Tv1Zzye_2QzT/view?usp=sharing', 'dates.zip')
zipPath <- '/downloads/dates.zip'
dir.create('planting_dates')
out <- '/downlaods/planting_dates/maize_usa_crop_calendar.nc'
unzip(zipPath, out)

## 2. Downloading and Processing Corn Yield Data from USDA Quickstats Database

I accessed the USDA yield data directly from a web browser and the steps involved are described below:
- Go to the USDA Quickstats database page at https://quickstats.nass.usda.gov/
- Under the 'Select Commodity' section select the parameter mentioned for each category - Program: SURVEY -> Sector: CROPS -> Group: FIELD CROPS -> Commodity: CORN -> Data Item: CORN, GRAIN - YIELD, MEASURED IN BU/ACRE
- Under the 'Select Location" section select 'COUNTY' under Geographic Level
- Under the 'Select Time' section select years from 1960 to 1974
- After the data table is loaded click on 'spreadsheet' on the top right 
- Repeat the above steps with years 1975 to 1994 and again with years 1995 to 2020. This is necessary since not more than 50,000 records can be retrieved at once.

Note that the API available for accessing USDA Quickstats data (https://quickstats.nass.usda.gov/api/) is basic and can only retrieve summary statistics of datasets.  

### 2.1 Mapping Yields with Counties

In [ ]:
# Read in the csv files
df1 <- read.csv('Corn_1960-1974_yield_Bu_per_acre.csv')[,c(2,6,7,11,20)]
df2 <- read.csv('Corn_1975-1994_yield_Bu_per_acre.csv')[,c(2,6,7,11,20)]
df3 <- read.csv('Corn_1995-2020_yield_Bu_per_acre.csv')[,c(2,6,7,11,20)]

# Merge csv files to create a continuous time series
df <- rbind(df1,df2,df3)
df <- df[!is.na(df$County.ANSI),]
df$State.ANSI <- sprintf("%02d", as.numeric(df$State.ANSI)) # to make '1' as '01' - this is to match with the state FIPS code in the shapefile
df$County.ANSI <- sprintf("%03d", as.numeric(df$County.ANSI)) # to make '1' as '001' - this is to match with the state FIPS code in the shapefile
df$geoid <- paste0(df$State.ANSI,df$County.ANSI) # create a new column to merge with shapefile

# Map yields with counties

library(sf)
library(raster)

sp.df <- split.data.frame(df,df$Year) # split yield data into a list with years as indices

st <- st_read('/downloads/gz_2010_us_050_00_20m.shp') # read shapefile
st <- st[!(st$STATE %in% c('02','15','72')),] # to subset sf object without AK,HI and PU states


for(i in 1:length(sp.df)){
    yld_geo <- as.data.frame(sp.df[[i]][,c(5,6)])
    colnames(yld_geo) <- c('yield','geoid')
    match <- match(substr(st.df$GEO_ID,10,14),yld_geo$geoid)
    
    yield <- rep(NA, 3109)
    for(j in 1:3109){
        if(is.na(match[j])){
            yield[j] <- NA
        }else{yield[j] <- yld_geo$yield[match[j]]} # to write match values - for eg. if match[2] = 22, then yield[2] <- usda$Value[22]
    }

    yield <- gsub(',','',yield) #remove comma becasue as.numeric won't work with comma

    yield.df <- as.data.frame(cbind(yield,st.df$GEO_ID))
    colnames(yield.df) <- c(paste0(i+1959,'_yd'),'GEO_ID')
    
    yield.df[,1] <- as.numeric(yield.df[,1])
    yield.df[,1] <- 67.251*yield.df[,1] # convert Bu/Acre to Kg/ha
    
    st <- merge(st,yield.df, by = 'GEO_ID')
}

# write a shapefile (analogus to geopanda dataframe)
st_write(st, 'Corn_1960-2020_yield_kg_per_ha.shp', driver = "ESRI Shapefile") 


# Rasterize yield to 50 km gridded format and create a stack of all years
# rasterizing to 50 km grids to match the resolution of data on planting and harvesting dates 

shp <- st_read('/downloads/Corn_1960-2020_yield_kg_per_ha.shp')
ref <- st_read('/downloads/Corn_2010_hectares_harvested.shp')
y <- raster(nrow = dim(shp)[1], ncol = dim(shp)[2])
for(i in 1960:2020){
    ras <- raster(res = c(0.5,0.5), extent(ref), crs = crs(ref))
    field <- paste0('X',i,'_yd_x')
    ras <- rasterize(shp, ras, field)
    y <- addLayer(y,ras)
    rm(ras)
}

saveRDS(y, '/downloads/corn_yield_stack_1960_2020.rds') # write raster stack as an RDS file - smaller fize size and faster processing


### 2.2 Detrending Yield 
To account for time fixed effects, the yield data is detrended by removing the time trends and retaining anomalies.

In [4]:
# to identify trends and get the slope and intercept for each pixel

y <- readRDS('/downloads/corn_yield_stack_1960_2020.rds')
yd <- as.array(y)

dim(yd) # get dimensions of array

slope <- raster(nrow = 50, ncol = 116) # 50 and 116 are dimensions of the array
int <- raster(nrow = 50, ncol = 116)
l <- 1:61 # time in years
for(i in 1:50){
    for(j in 1:116){
            vector <- as.numeric(yd[i,j,])
            vector[vector == 0] <- NA
            if(all(is.na(vector))){
                    slope[i,j] <-  NA
                    int[i,j] <- NA
            } else{
            fit <- lm(vector~l)
            slope[i,j] <-  fit$coefficients[2]
            int[i,j] <- fit$coefficients[1]       
        }
    }
}


# remove trends and write the detrended yields to a raster brick

det <- array(numeric(), c(50,116,61))

for(i in 1:50){
    for(j in 1:116){
        for(k in 1:61){
            det[i,j,k] <- (yd[i,j,k] - (slope[i,j]*k + int[i,j]))
        }
    }
}

det.ras <- brick(det, xmn = extent(y)[1], xmx = extent(y)[2], ymn = extent(y)[3], ymx = extent(y)[4], crs = crs(y))
saveRDS(det.ras, 'detrended_yield_stack_1960_2020.rds')

## 3. Downloading and Processing ERA5 Reanalysis Data

ERA5 data is hosted on the Climate Data Store (CDS) database, maintained by ECMWF. All data on this server can be accessed through a python-based API client. The 'cdsapi' package on python is necessary for accessing data. Documentation and tutorials can be accesed at https://cds.climate.copernicus.eu/api-how-to 

After creating an account and registering for API key, run the below python code:

In [56]:
# downloading hourly maximum surface temperature through API

import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'year': [
            '1960', '1961', '1962',
            '1963', '1964', '1965',
            '1966', '1967', '1968',
            '1969', '1970', '1971',
        ],
        'variable': 'maximum_2m_temperature_since_previous_post_processing',
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            50, -126, 24,
            -64,
        ],
    },
    'tmax_hourly_1960.nc')

Since a maximum of 120,000 records can be retrieved at once and also because yearly files are easier to calculate annual growing season KDD and GDD, re-run the above code by updating the 'year' section. This can also be done with a simple 'for' loop iterated over years. Once all data is downloaded for maximum temperature, repeat the above process for other variables by updating variable name in the 'variable' section, as listed below:

- Minimum surface temperature: 'maximum_2m_temperature_since_previous_post_processing'
- Soil moisture: 'volumetric_soil_water_layer_1'

I used Climate Data Operators (CDO), a set of tools for processing and analyzing all forms of gridded data, for creating a complete timeseries from 1960 to 2020 for each variable, followed by calculation of daily averages from hourly values. Steps for installing and using CDO is available here - https://code.mpimet.mpg.de/projects/cdo/wiki/Tutorial

### 3.1 Processing Data in CDO

Use the following CDO commands in a linux terminal for calculating daily average:

>  cdo daymean  tmax_hourly_1960.nc  tmax_daily_1960.nc

>  cdo daymean  tmin_hourly_1960.nc  tmin_daily_1960.nc

>  cdo daymean  sm_hourly_1960.nc  sm_daily_1960.nc

The above commands can be automated for all years in a bash script as shown below:

In [ ]:
#!/bin/sh

for i in $(seq 1960 2020);
do
  screen -dmS "$i" cdo daymean tmax_hourly_$i.nc tmax_daily_$i.nc
  sleep 2s
done    

The next step is to regrid all netCDF files to 0.5 by 0.5 degrees to match the resolution of the planting and harvesting dates and yield data. For this, create a text file named 'grid.txt' and enter the below lines:

>gridtype  = lonlat

>gridsize  = 6448

>xsize     = 124

>ysize     = 52

>xname     = longitude

>xlongname = "longitude"

>xunits    = "degrees_east"

>yname     = latitude

>ylongname = "latitude"

>yunits    = "degrees_north"

>xfirst    = -125.749992370605

>xinc      = 0.5

>yfirst    = 49.7500012715658

>yinc      = -0.500000012466331

>scanningMode = 64


Finally, run the below cdo commands to complete regridding:

> cdo remapbil,grid.txt tmax_daily_1960.nc tmax_daily_1960_50km.nc

> cdo remapbil,grid.txt tmin_daily_1960.nc tmin_daily_1960_50km.nc

> cdo remapbil,grid.txt sm_daily_1960.nc sm_daily_1960_50km.nc

These commands can also be automated to run over all years using a bash script as shown above.


### 3.2 Calculating Killing Degree Days (KDD) and Growing Degree Days (GDD)

Growing season KDD and GDD is calculated over each pixel, using the planting and harvesting dates. Below is an R code for calculating and writing annual growing season KDD and GDD into netCDF files.

In [ ]:
# To calculate and write Growing Season KDD into netCDF files 

library(ncdf4) # for netCDF file handling

nc <- nc_open('/downloads/maize_usa_crop_calendar.nc')

# plantting and harvest dates - GROWING SEASON
pl <- ncvar_get(ma, 'plant.start')
hr <- ncvar_get(ma, 'harvest.end')

for(i in 1960:2020){
    nc <- nc_open(paste0('/downloads/tmax_daily_',i,'.nc'))
    
    lon.ar <- as.array(ncvar_get(nc, 'longitude'))
    lat.ar <- as.array(ncvar_get(nc, 'latitude'))
    
    temp <- ncvar_get(nc, 'mx2t') # read the maximum temperature array
    temp <- temp - 273.15     # convert temperature from Kelvin scale to degrees celcius
    temp <- temp - 29         # definition of KDD
    temp[temp < 0] <- 0
    
    kdd <- array(numeric(), c(124,52)) #dimension of the temperature array
    
    for(x in 1:124){
    for(y in 1:52){
        if(!is.na(pl[x,y])){
            kdd[x,y] <- sum(temp[x,y,(pl[x,y]:hr[x,y])], na.rm = T)
        } else{
            kdd[x,y] <- NA
            }
        }
    }
    
    ncfname <- paste0('/downloads/kdd_gs_corn_',i,'.nc') # growing season KDD
    dname <- "kdd" 

    londim <- ncdim_def("lon","degrees_east",as.double(lon.ar)) 
    latdim <- ncdim_def("lat","degrees_north",as.double(lat.ar)) 

    #tunits <- 'hours since 31-12-2020 23:00:00 IST'
    timedim <- ncdim_def("time",'static',1)

    # define variables
    dlname <- "Killing degree days"
    kdd_def <- ncvar_def("kdd","deg_C_days",list(londim,latdim,timedim),-999,dlname,prec="single")
    
    ncout <- nc_create(ncfname,list(kdd_def),force_v4=TRUE)

    # put variables
    ncvar_put(ncout,kdd_def,kdd)
    
    nc_close(ncout)
    nc_close(nc)
}

In [ ]:
# To calculate and write Growing Season GDD into netCDF files 

library(ncdf4)
nc <- nc_open('/downloads/maize_usa_crop_calendar.nc')

# plantting and harvest dates - GROWING SEASON
pl <- ncvar_get(ma, 'plant.start')
hr <- ncvar_get(ma, 'harvest.end')


for(i in 1960:2020){
    max <- nc_open(paste0('/downloads/tmax_daily_',i,'.nc'))
    min <- nc_open(paste0('/downloads/tmin_daily_',i,'.nc'))
    
    lon.ar <- as.array(ncvar_get(max, 'longitude'))
    lat.ar <- as.array(ncvar_get(max, 'latitude'))
    
    tmax <- ncvar_get(max, 'mx2t')
    tmax <- tmax - 273.15
    tmax[tmax > 30] <- 30
    
    tmin <- ncvar_get(min, 'mn2t')
    tmin <- tmin - 273.15
    tmin[tmin < 10] <- 10
    
    gdd <- ((tmax+tmin)/2 - 10) # Definition of GDD
    
    gdd.ar <- array(numeric(), c(124,52))
    
    for(x in 1:124){
    for(y in 1:52){
        if(!is.na(pl[x,y])){
            gdd.ar[x,y] <- sum(gdd[x,y,(pl[x,y]:hr[x,y])], na.rm = T)
        } else{
            gdd.ar[x,y] <- NA
            }
        }
    }
    
    ncfname <- paste0('/downloads/gdd_gs_corn_',i,'.nc') # growing season GDD
    dname <- "gdd" 

    londim <- ncdim_def("lon","degrees_east",as.double(lon.ar)) 
    latdim <- ncdim_def("lat","degrees_north",as.double(lat.ar)) 

    #tunits <- 'hours since 31-12-2020 23:00:00 IST'
    timedim <- ncdim_def("time",'static',1)

    # define variables
    dlname <- "Growing degree days"
    gdd_def <- ncvar_def("gdd","deg_C_days",list(londim,latdim,timedim),-999,dlname,prec="single")
    
    ncout <- nc_create(ncfname,list(gdd_def),force_v4=TRUE)

    # put variables
    ncvar_put(ncout,gdd_def,gdd.ar)
    
    nc_close(ncout)
    nc_close(max)
    nc_close(min)
}

### 3.2 Calculate Growing Season Average Soil Moisture 

In [ ]:
# to write out daily growing season soil moisture

for(i in 1960:2020){
    nc <- nc_open(paste0('/downloads/sm_daily_',i,'.nc'))
    
    lon.ar <- as.array(ncvar_get(nc, 'longitude'))
    lat.ar <- as.array(ncvar_get(nc, 'latitude'))
    
    sm <- ncvar_get(nc, 'swvl1')
        
    kdd <- array(numeric(), c(124,52))
    
    for(x in 1:124){
    for(y in 1:52){
        if(!is.na(pl[x,y])){
            kdd[x,y] <- mean(sm[x,y,(pl[x,y]:hr[x,y])], na.rm = T)
        } else{
            kdd[x,y] <- NA
            }
        }
    }
    
    ncfname <- paste0('/downloads/sm_corn_gs_mean_',i,'.nc') # growing season soil moisture
    dname <- "sm"  
    
    londim <- ncdim_def("lon","degrees_east",as.double(lon.ar)) 
    latdim <- ncdim_def("lat","degrees_north",as.double(lat.ar)) 

    #tunits <- 'hours since 31-12-2020 23:00:00 IST'
    timedim <- ncdim_def("time",'static',1)

    # define variables
    dlname <- "Growing Season Average Soil Moisture - Volumetric Soil Water(m^3/m^3)"
    sm_def <- ncvar_def("sm","deg_C_days",list(londim,latdim,timedim),-999,dlname,prec="single")
    
    ncout <- nc_create(ncfname,list(sm_def),force_v4=TRUE)

    # put variables
    ncvar_put(ncout,sm_def,sm)
    
    nc_close(ncout)
    nc_close(nc)
}